<a href="https://colab.research.google.com/github/zhangling297/MAT_CS_599_deepLearningClassPracitices/blob/main/Copy_of_Fully_connected_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Building convolutional Neural Networks from Scratch

fully- connected NN for binary classification - using heart disease data - upload the data, subset the columns to

In [ ]:
from IPython.core.interactiveshell import default_banner
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
# When training Deep Learning models, randomness enters the process in a few different places

  # 1.starting values for the weights(the opitmizer will try to improve these weights)
  # 2. The order in which we process the minibatches when we do SGD
  # 3. When we split the data into Train, Validation, Test etc
  # 4. Dropout(if using regularization)

# Set the seed for different random number generators so that the results will be the same each time  the notebook is run

In [ ]:
keras.utils.set_random_seed(42)

In [ ]:
df = pd.read_csv('/content/heart_disease.csv')

In [ ]:
df.shape

In [ ]:
df.head()

# check 1's and 0 's to see if they are balanced.

In [ ]:
df.target.value_counts(normalize=True, dropna=False)

Preprocessing # Collet groups variables into two lists

In [ ]:
categorical_Variables = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'ca', 'thal']
numerics = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope']



*   one-hot encode the categorical variables with the pandas get_dummies function

*   Normalize the numeric varibles



In [ ]:
df = pd.get_dummies(df, columns = categorical_Variables)

In [ ]:
df.head()

NNs work best when the inputs are all roughly in the same range. So standard practice is to standardize the numeric varibles; Split the data into 80% training and 20% testing set before normalization

In [ ]:
test_df = df.sample(frac=0.2, random_state=42)
train_df = df.drop(test_df.index)

In [ ]:
train_df.shape

In [ ]:
test_df.shape

Calculate the mean and standard deviation of every numeric variable in the training set.

In [ ]:
means = train_df[numerics].mean()
sd = train_df[numerics].std()

In [ ]:
means

Standardize the train and test dataframes with these means and standard deviations

In [ ]:
train_df[numerics] = (train_df[numerics] - means)/sd

In [ ]:
test_df[numerics] = (test_df[numerics]- means)/sd

In [ ]:
train_df.head()

So at this point, the whole dataset is numeric. Now feed data to keras/Tensorflow is as Numpy arrays so to convert our two dataframes to Numpy arrays

In [ ]:
train = train_df.to_numpy()
test = test_df.to_numpy()

Final step, features X and dependent variable y are both inside the train and test arrays so to seperate them out as the target column is y variable (counting from 0). The np.delete function is for selecting all columns (total 6) except one.

In [ ]:
train_X = np.delete(train, 6, axis=1).astype(np.float32)
test_X = np.delete(test, 6, axis=1).astype(np.float32)

In [ ]:
train_X.shape, test_X.shape

Next, Select just the 6th column and define the train and test y variables.

In [ ]:
train_y = train[:, 6].astype(np.float32)
test_y = test[:, 6].astype(np.float32)

In [ ]:
train_y.shape, test_y.shape

# Build a model

*   Define model in Keras ## Start with a single hidden layer; Since this is a binary flassification problme, we will use a signmoid activation in the output layer
*   



In [ ]:
num_columns = train_X.shape[1]
#define the input layer
input = keras.Input(shape=(num_columns,))

#feed the input vector to the hidden layler (can give names to each layer to help)
#keep track. This doesn't affect the training
h = keras.layers.Dense(16, activation='relu', name="Hidden")(input)

#feed the output of the hidden layer to the output layer
output = keras.layers.Dense(1, activation='sigmoid', name='Output')(h)

# tell Keras taht this (input, output) pair is your model
model = keras.Model(input, output)

# use model.command to get a quick overview of what have been defined

In [ ]:
model.summary()

(28 +1) *16 + (16 +1) *1

Visualize the network graphically as well using Keras' plot_model function

In [ ]:
keras.utils.plot_model(model, show_shapes=True)

# Set Optimization Parameters

Summary: the model is defined, now need to tell Keras theree things:


*   What loss function to use- output variable is binary, we can select the binary_crossentropy loss function
*   Which optimizer to use - we will use a sibling of SGD called Adam which which is an excellent default choice
*  What metrcs you want to report out_ in classfication problems accuracy is the metric



In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Training the model

To kickoff training, 3 things have to be decided:

1.   The bach size (32 is the default)
2.   The number of epochs (i.e.., how many passes through the training data - usually 20-30 passes is a good starting point)
3. whether uses a validation set. This decision will be good for overfitting detection and regularization via early stopping so we will ask Keras to automatically use 20% of the data points as a validation set



**use model.fit to train the model**

Store the output of the training process in history. It will make it easy later to investigate what happened during training

In [ ]:
history = model.fit(train_X, #the array with the input X columns
                    train_y, #the array witht he output y column
                    epochs=300, #numberj of epochs to run
                    batch_size=32, # number jof samples (ie data points) per batch
                    verbose=1, # verbosity during training)
                    validation_split=0.2) # use 20% of the data for validation

Plotting metrics like loss and accuracy as a function of the # of epochs is a good way to understand how training has progressed.

In [ ]:
history_dict = history.history
print(history_dict.keys())

In [ ]:
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Evaluate the model to see how the model does on the test set

model.evaluate is a function used to calcualte the performance of your model on any dataset

In [ ]:
model.evaluate(test_X, test_y)

### Predicting new data with the model

How to save a Keras model and use it for prediction  

SUmmary : pre-processing (one-hot encoding and normalization) - remember what pre-processing we did and carry that information (mean and variance of each variable) along with the model to correctly use the model in the future

using Keras preprocessing layers.

# New Section

In [ ]:
his

In [ ]:
Training the model

# New Section

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

If you see output indicating that a GPU is available, then TensorFlow is configured to use it. When you run your deep learning models, they should automatically leverage the GPU for faster computations.